<a href="https://colab.research.google.com/github/jbardsle/URSP688YFinal/blob/main/FinalProjectPaperVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import geopandas as gpd
import os
import numpy as np
import matplotlib.pyplot as plt
import shapely
import os


In [ ]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/FinalProject')

In [ ]:
OppSites = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/FinalProject/OpportunitySites2/OpportunitySites2.shp')

In [ ]:
# This begins the process to clean the original large data files. For loading speed,
# the smaller versions are loaded below, and this section is commented out.
# OppSites is small enough that it is loaded below.
# PgParcels = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/FinalProject/Property_Info_Py (1)/Property_Info_Py.shp')

In [ ]:
# Demogs = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/FinalProject/PG_Blocks/PG_Blocks.shp')

In [ ]:
#Select the opportunity sites that have a 3 mile radius completely withing Prince George's County
#and a small sliver of DC
OppSites = OppSites[OppSites['DissolveID'].isin([11,31,188,])]

In [ ]:
#Convert all geodataframes to UTM-18 for distance measurement
# PgParcels = PgParcels.to_crs(epsg = 32618)
OppSites = OppSites.to_crs(epsg = 32618)
# Demogs = Demogs.to_crs(epsg = 32618)

In [ ]:
#Create a buffer around each site of approximately three miles (4828 meters) to represent the market area
OppSitesBuffer = OppSites['geometry'].buffer(4828)

In [ ]:
#Copy the Opportunity Sites layer so that it can be replaced with the buffer geometry
OppSitesWithBuffer = OppSites.copy()

In [ ]:
#Replace original geometry with buffer geometry
OppSitesWithBuffer['geometry'] = OppSitesBuffer

In [ ]:
#plot the buffers with the opportunity sites
ax = OppSitesBuffer.plot()
OppSites.plot(ax=ax, color = 'red')

In [ ]:
# #Create a single shape to represent the maximum area of interest (3 miles from the three sites)
# MaxArea = OppSitesBuffer.unary_union
# #Convert this polygon object to a geodataframe
# MaxArea_series = gpd.GeoSeries(MaxArea)
# MaxAreaGDF = gpd.GeoDataFrame(geometry = MaxArea_series, crs = 32618)

In [ ]:
# PgParcelsSmall = gpd.sjoin(PgParcels, MaxAreaGDF,how="inner", op='intersects')

In [ ]:
# DemogsSmall = gpd.sjoin(Demogs, MaxAreaGDF,how="inner", op='intersects')

In [ ]:
#Load PgParcelsSmall from file (rather than performing the above process)
PgParcelsSmall = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/FinalProject/SmallerFiles/PgParcelsSmall3Miles/PgParcelsSmall3Mile.shp')

In [ ]:
#Load DemogsSmall from file (rather than performing the above process)
DemogsSmall = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/FinalProject/SmallerFiles/DemogsSmall3Miles/DemogsSmall3Mile.shp')

In [ ]:
#Remove all columns except the one containing the Geoid, total pop, hispanic pop, and hispanic percent
DemogsSmall = DemogsSmall[['GEOID','P0010001','P0020002','PCT_P00200','geometry']]

In [ ]:
#Rename columns for ease of use
DemogsSmall.rename(columns={'P0010001': 'Total_Pop', 'P0020002': 'Hisp_Pop', 'PCT_P00200': 'Hisp_Pct'}, inplace=True)

In [ ]:
#Get rid of index_right column in PgParcelsSmall which causes an error in a future join
PgParcelsSmall.drop('index_righ', axis=1, inplace=True)

In [ ]:
#JoinPgParcelsSmall with demographic data
ParcelDemogs = gpd.sjoin(PgParcelsSmall,DemogsSmall,how="inner", op = "intersects")

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [ ]:
# Group by the parcel account number so that there is one record per parcel
ParcelDemogsCln = ParcelDemogs[['ACCT_PRIMA','DUS','GEOID','Total_Pop','Hisp_Pop','Hisp_Pct','geometry']].groupby('ACCT_PRIMA').first()

In [ ]:
#a function to ensure that there are no nulls and that percents are in
#decimal format
def CleanPcts(input):
  input['Hisp_Pct2'] = None
  input.loc[(input['Hisp_Pct'] == 0) | (input['Hisp_Pct'].isnull()), 'Hisp_Pct2'] = 0
  input.loc[input['Hisp_Pct'] > 0, 'Hisp_Pct2'] = input['Hisp_Pct'] / 100
  return input


In [ ]:
#Running the CleanPcts function
CleanPcts(ParcelDemogsCln)

In [1]:
#A function to calculate the distance between origin and destination
#parcels and calculate the decay in number of households associated with those parcels
#based on the distance and the selected exponent (1 = linear, 2 = quadratic, etc)
def dist_decay(origins, dest, exp):
  #a dictionary to contain the lists of distances from each polygon
  dict_of_lists = {}
  dict_of_lists_hisp = {}
  for polygon in origins['geometry']:
    #lists to contain the distances from each origin parcel, the number of dwelling units,
    #and the percent Hispanic for each destination parcel
    dist_list = []
    DU_list = []
    HispPct_list = []
    #appends the values for the destination parcel to the lists for each destination
    #parcel
    for point, du, pct in zip(dest['geometry'], dest['DUS'], dest['Hisp_Pct2']):
      dist_list.append(point.distance(polygon))
      DU_list.append(du)
      HispPct_list.append(pct)
    #creates new lists for the weighted values
    dist_list_w = []
    HispPct_w = []
    #locations less than 0.25 miles/402m away will have the value 1
    #while others will decay
    for dist, du, pct in zip(dist_list, DU_list, HispPct_list):
      if dist <= 402:
        dist_list_w.append(1*du)
        HispPct_w.append(1*du*pct)
      elif dist > 402:
        dist_list_w.append(((402/dist)**exp)*du )
        HispPct_w.append(((402/dist)**exp)*du*pct)
    #summing the values for each origin polygon
    dist_list_w_sum = sum(dist_list_w)
    HispPct_w_sum = sum(HispPct_w)
    #creating a new dictionary key, and adding the sum
    #from above as the value
    dict_of_lists[polygon] = dist_list_w_sum
    dict_of_lists_hisp[polygon] = HispPct_w_sum
    #creating a new column that maps the results of the
    #dictionary above by the geometry column
  origins['Customers'] = origins['geometry'].map(dict_of_lists)
  origins['HispCustomers'] = origins['geometry'].map(dict_of_lists_hisp)
  #returns the original table with the new column
  return origins

In [ ]:
#Running the dist_decay function
dist_decay(OppSites,ParcelDemogsCln,2)

In [ ]:
#creating a percent column
OppSites['PctHisp'] = (OppSites['HispCustomers'] / OppSites['Customers'])

In [ ]:
#checking the results
OppSites.head()

,DissolveID,DissolveCo,DissolveCi,DissolveAd,DissolveAr,DissImpVal,DissLandVa,DissImpV_1,Dissolve_O,DissMinYea,...,COUNT_Ch_2,Local_Pct,Chain_Pct,Shape_Leng,Shape_Area,Incentiv_1,geometry,Customers,HispCustomers,PctHisp
1,11.0,1,LANHAM,7591 ANNAPOLIS RD,4.064821,0,0,0.000000,CARMELA PROPERTIES LLLP,1972,...,2,0.000000,1.000000,1852.864568,177063.602279,None,"POLYGON ((336805.276 4313016.260, 336796.608 4...",3381.254147,1061.587270,0.313963
2,31.0,2,HYATTSVILLE,7520 ANNAPOLIS RD,10.008090,0,0,0.000000,COUNTY CENTER JOINT VENTURE,1961,...,10,0.600000,0.400000,2714.770006,435952.390015,None,"POLYGON ((336530.388 4312971.656, 336515.614 4...",3242.822139,1208.643190,0.372713
6,188.0,1,RIVERDALE,5603 KENILWORTH AVE,10.916297,7614367,4791600,1.589107,RIVERDALE PLAZA SHP CTR LTD PTSP,1968,...,14,0.714286,0.285714,2844.545113,475513.904654,None,"POLYGON ((333714.344 4313969.197, 333721.553 4...",4503.489949,2666.262853,0.592044


In [ ]:
# Exporting the results to csv
# OppSites.to_csv('/content/drive/MyDrive/Colab Notebooks/FinalProject/Results/May10Exponential2.csv')